In [1]:
from researcher import Researcher
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [2]:
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8042
WEBHOOK_BASE = ""
ADMIN_URL = "http://researcher-agent:8041"
# ADMIN_URL = "http://0.0.0.0:8041"

agent_config = {
    "webhook_host": WEBHOOK_HOST,
    "webhook_port": WEBHOOK_PORT,
    "webhook_base": WEBHOOK_BASE,
    "admin_url": ADMIN_URL
}



In [3]:
import torch

# models
from torch import nn
from torch import optim
from torch.autograd import Variable
import os
import sys

from opacus import PrivacyEngine

# The Researcher generates the model

model = nn.Sequential(
            nn.Linear(8, 4),
            nn.Sigmoid(),
            nn.Linear(4, 2),
            nn.Sigmoid(),
            nn.Linear(2, 1),
            nn.Sigmoid()
)

In [4]:
validation_data_path = 'data/coordinator_validation_data.csv'

In [5]:
researcher = Researcher(agent_config, validation_data_path, model)

CLEANING THE VALIDATION SET
VALIDATION SET HAS BEEN CLEANED


## Request Authorised Research Credential

Copy invite from the [Health Research Authority Notebook](http://localhost:8889/notebooks/Health%20Research%20Regulator.ipynb)

In [ ]:
invite = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '48db8bb7-0543-4102-a391-1c4f1ed7dfde', 'label': 'NHS Health Research Authority', 'recipientKeys': ['5GbdLanjtXFE5anxYHwTAy9iAZm1bgLoxpz1va3xPmfC'], 'serviceEndpoint': 'http://172.17.0.1:8030'}

In [ ]:
await researcher.agent_controller.connections.accept_connection(invite)

## Configure Authentication Policy

Note: Normally we would specify the credential definition here specifying which entities we trust to issue this schema, but this changes every time because it's a demo.

Feel free to copy yours across from the NHS Trust though.

In [6]:


schema_id = "BvRKFu1fdzGdJzQcKb8e8p:2:NHS Hosptial:0.0.1"
# cred_def = "WfntKNFwXMQfgmU9ofbxPM:3:CL:156569:default"

revocation = False
exchange_tracing = False

#Enable this to ask for attributes to identity a user
#TODO - change restriction to schemaId or credentialId

req_attrs = [
    {"name": "name", "restrictions": [{"schema_id": schema_id}]},#, "cred_def_id": cred_def}]},
]

global indy_proof_request
indy_proof_request = {
    "name": "Proof of Hospital",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
#         f"0_{req_pred['name']}_GE_uuid":
#         req_pred for req_pred in req_preds
    },
}



In [7]:
researcher.set_authentication_policy(indy_proof_request)

## Send Invite to Hospital 1

Option: We could make this a multiuse invite.

In [8]:
researcher.new_dataowner_invite()

Connection Handler Called
Connection 3ea74793-7ae8-4d8c-a596-95f04ba14feb in State invitation

♫♫♫ > STEP 1: Send the aries invitation to the data owner

{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "df76459d-7a65-462e-ae5d-bc1f96a7674f", "serviceEndpoint": "http://172.17.0.1:8040", "label": "Health Researcher", "recipientKeys": ["9QkEnGGQHUNp95rRdsf2GxyGzGniuApqGmoQuXT5UvCC"]}

Connection Handler Called
Connection 3ea74793-7ae8-4d8c-a596-95f04ba14feb in State request
Accepting request  3ea74793-7ae8-4d8c-a596-95f04ba14feb
Connection Handler Called
Connection 3ea74793-7ae8-4d8c-a596-95f04ba14feb in State response
Sending trust ping 3ea74793-7ae8-4d8c-a596-95f04ba14feb
Connection Handler Called
Connection 3ea74793-7ae8-4d8c-a596-95f04ba14feb in State active
Pending dataowner connection moved to active. Challenging with auth policy if set
Handle present proof
Role verifier, Exchange 09167e7a-a761-46a0-ad35-31dc25725a89 in state request_sent
Handle pr

## Send Invite to Hospital 2

In [10]:
researcher.new_dataowner_invite()

Connection Handler Called
Connection bfd3dc93-e8bb-4562-9785-be2d487bebcd in State invitation

♫♫♫ > STEP 1: Send the aries invitation to the data owner

{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "e3165dfa-535e-4004-a64d-46b3eed5c90f", "serviceEndpoint": "http://172.17.0.1:8040", "label": "Health Researcher", "recipientKeys": ["AWeZ4ZRfZnEHwhxHFFxJWE2JkdVmw9CCRnWDRSWVH2nF"]}

Connection Handler Called
Connection bfd3dc93-e8bb-4562-9785-be2d487bebcd in State request
Accepting request  bfd3dc93-e8bb-4562-9785-be2d487bebcd
Connection Handler Called
Connection bfd3dc93-e8bb-4562-9785-be2d487bebcd in State response
Sending trust ping bfd3dc93-e8bb-4562-9785-be2d487bebcd
Connection Handler Called
Connection bfd3dc93-e8bb-4562-9785-be2d487bebcd in State active
Pending dataowner connection moved to active. Challenging with auth policy if set
Handle present proof
Role verifier, Exchange 876b5ba5-5de5-47f8-92e6-98627e3805ae in state request_sent
Handle pr

## Send Invite to Hospital 3

In [ ]:
researcher.new_dataowner_invite()

## Initiate Learning

PPML Learning messages are sent over DIDComm using the basic message protocol. You will be able to see the encrypted message in the terminal you initialised this project.

In [11]:
researcher.initiate_learning()

MODEL OPENED FOR TRANSPORT
Training model with 2 DataOwners
Still learning
Continue Learning bfd3dc93-e8bb-4562-9785-be2d487bebcd
HOSPITAL MODEL LOADED

PRINTING PARAMETERS:


Sequential(
  (0): Linear(in_features=8, out_features=4, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=4, out_features=2, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=2, out_features=1, bias=True)
  (5): Sigmoid()
)


0.weight tensor([[-0.0071,  0.9379, -0.1551,  1.0665,  0.1179,  0.5294, -0.5972, -3.4125],
        [-0.4067, -2.0534,  1.7080, -1.6800,  0.3804,  2.2666, -0.4622,  0.8771],
        [ 0.9054, -1.2353,  0.4089,  3.3475,  0.0547, -1.5822,  0.4324, -1.7091],
        [ 0.0552,  0.1231, -0.0514,  1.2591,  0.3860, -0.8143, -1.4579,  3.4421]])
0.bias tensor([ 1.7992, -2.8332, -0.8909, -1.1692])
2.weight tensor([[-3.4769,  2.6372,  2.3429,  2.3349],
        [ 2.4076, -2.0967, -1.6529, -1.4950]])
2.bias tensor([-2.7266,  1.7754])
4.weight tensor([[ 6.8013, -4.5238]])
4.bias tensor([-1.1125])
